In [108]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from collections import defaultdict
import time

In [7]:
glass=pd.read_csv('glass.csv',sep=',') #Чтение датасета

In [9]:
glass.describe()#Описательная статистика датасета

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
count,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000
mean,1.518365,13.407850,2.684533,1.444907,72.650935,0.497056,8.956963,0.175047,0.057009,2.780374
std,0.003037,0.816604,1.442408,0.499270,0.774546,0.652192,1.423153,0.497219,0.097439,2.103739
min,1.511150,10.730000,0.000000,0.290000,69.810000,0.000000,5.430000,0.000000,0.000000,1.000000
25%,1.516522,12.907500,2.115000,1.190000,72.280000,0.122500,8.240000,0.000000,0.000000,1.000000
50%,1.517680,13.300000,3.480000,1.360000,72.790000,0.555000,8.600000,0.000000,0.000000,2.000000
75%,1.519157,13.825000,3.600000,1.630000,73.087500,0.610000,9.172500,0.000000,0.100000,3.000000
max,1.533930,17.380000,4.490000,3.500000,75.410000,6.210000,16.190000,3.150000,0.510000,7.000000


In [11]:
glass.info()#Проверка на пропуски

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      214 non-null    float64
 1   Na      214 non-null    float64
 2   Mg      214 non-null    float64
 3   Al      214 non-null    float64
 4   Si      214 non-null    float64
 5   K       214 non-null    float64
 6   Ca      214 non-null    float64
 7   Ba      214 non-null    float64
 8   Fe      214 non-null    float64
 9   Type    214 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 16.8 KB


In [19]:
def standartize_data(data): #Стандартизация данных
    d=(data-data.mean())/data.std()
    return d.to_numpy()


In [13]:
X=glass[['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe']]

In [42]:
y=glass[['Type']]

In [21]:
x=standartize_data(X)

In [23]:
pca = PCA(n_components=2)
Y=pca.fit_transform(x)

In [146]:
#plt.scatter(Y[:,0],Y[:,1])
#plt.show()

In [72]:
def euclide(x_t,x):
    return np.sum(np.power((x_t[:,None]-x),2),axis=-1)

def gauss_kernel(dists, k):#Расчёт Гауссова ядра
    return 1 / np.sqrt(2*np.pi)*np.exp(-((dists[:,:k]/dists[:,k].reshape(dists.shape[0],-1)**2)/2))
    
def abs_dist(x_t,x): #L1-метрика
    return np.sum(np.abs(x_t[:,None]-x),axis=-1)

In [138]:
class KNN():
    def fit_predict(self, x_train, y_train, x_test, k, method='euclide'):
        if method=='euclide':
            dists=euclide(x_test, x_train)#Расстояния от вводимых точек до обучающих
        else:
            dists=abs_dist(x_test, x_train)
        s_dists=np.sort(dists)[:,:k+1]#Отсортированные k ближайших к вводимым обучающих точек 
        s_idx=np.argsort(dists)[:,:k]#Индексы отсортированных точек
        weights=gauss_kernel(dists,k)#Веса объектов
        marks=[]#Итоговые метки классов
        for i in range(s_idx.shape[0]): #Итерируемся по каждому тестовому объекту
            u=y_train[s_idx[i,:k]] #Метки классов k ближайших точек
            data=defaultdict() 
            for j in range(k):#Объявление и заполнение словаря метка класса:суммарный вес
                data[u[j]]=0
            for j in range(k):
                data[u[j]]+=weights[i,j] #Добавление сумм весов в массив
            marks.append(max(data, key=data.get))#Итоговый ответ     
        return marks

    def leave_one_out(self,x,y, k, method='euclide'):
        marks=[]
        for i in range(x.shape[0]):#Подготовка данных и обучение
            x_test=np.array([x[i,:]])
            x_train=np.delete(x,i,axis=0)
            y_test=y[i]
            y_train=np.delete(y,i,axis=0)
            marks.append(self.fit_predict(x_train, y_train, x_test,k, method)[0])
        return np.sum(marks==y)/len(marks)#Возвращение точности классификации        

    def build_graph(self,x,y, method='euclide'):#Построение графика
        precisions=[]
        for i in range(1, 101):
            precisions.append(self.leave_one_out(x,y,i, method))
        plt.plot([i for i in range(1,101)],precisions)
        plt.xlabel('Число соседей')
        plt.ylabel('Точность классификации')
        plt.title('График эмпирического риска')
        plt.show()        

In [44]:
y=y.to_numpy().reshape(-1)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=2025)

In [76]:
knn=KNN()

In [150]:
#knn.build_graph(X_train,y_train)

In [154]:
#knn.build_graph(X_train,y_train, method='abs')

In [124]:
start = time.time()
predict_1=knn.fit_predict(X_train, y_train, X_test, 4, method='euclide')
end = time.time()
print('Accuracy:',np.sum(predict_1==y_test)/len(predict_1))

Accuracy: 0.7076923076923077


In [112]:
print("Время выполнения программы:",
      (end-start) * 10**3, "ms")

Время выполнения программы: 3.0260086059570312 ms


In [100]:
predict_2=knn.fit_predict(X_train, y_train, X_test, 1, method='abs')
print('Accuracy:',np.sum(predict_2==y_test)/len(predict_1))

Accuracy: 0.6615384615384615


In [140]:
neigh = KNeighborsClassifier(n_neighbors=4)#Эталонная реализация
start = time.time()
neigh.fit(X_train, y_train)
pr=neigh.predict(X_test)
end = time.time()

In [128]:
print("Время выполнения программы:",
      (end-start) * 10**3, "ms")

Время выполнения программы: 3.08990478515625 ms


In [130]:
np.sum(predict_1==pr)/len(predict_1) #Совпадение ответов на 86%

0.8615384615384616

In [142]:
print('Accuracy:',np.sum(pr==y_test)/len(pr)) #Точность одинаковая

Accuracy: 0.7076923076923077
